In [1]:
import random
import math

In [2]:
class Simple_Tabulation_Hash:
    def __init__(self, w, bits): #w = tamanho da tabela, c = qtd de partições, bits = tamanho da palavra
        self.c = 0
        self.bits = bits
        self.u = 2**bits-1
        self.w = w
        self.tab = [None for _ in range(self.w)]
        self.contador_elementos = 0
        self.contador_flags = 0

        if(self.bits == 64 or self.bits == 32):
            self.c = 8
        if(self.bits == 16 or self.bits == 4):
            self.c == int(math.sqrt(self.bits))
        if(self.bits == 8):
            self.c = 2
        if(self.bits == 2 or self.bits == 1):
            self.c = 1
        
        self.tabelas_aleatorias = []
        self.tamanho_tabela = 2**self.c
        
        random.seed(10)
        for i in range(0, self.c):
            T_i = []
            for i in range(self.tamanho_tabela):
                T_i.append(random.randint(0, self.u))

            self.tabelas_aleatorias.append(T_i)
    
    def particionar(self, x):
        x = int(x)
        binario_x = [int(i) for i in list('{0:0b}'.format(int(x)))]
        
        if(len(binario_x) < self.bits):
            binario_x = [0] * (self.bits - len(binario_x)) + binario_x
            
        binario_x = [binario_x[i:i + int(self.bits/self.c)] for i in range(0, len(binario_x), self.c)]

        return binario_x
            
        
    def funcao_hash(self, x):
        h_x = None
        binario_x = self.particionar(x)
        for indice, valor in enumerate(binario_x):
            if(indice > 0):
                h_x = h_x ^ self.tabelas_aleatorias[indice][sum(b<<indice for indice, b in enumerate(valor[::-1]))]
            elif(indice == 0):
                h_x = sum(b<<indice for indice, b in enumerate(valor[::-1]))
                
        return h_x
        
    def limpar(self):
        tabela_antiga = self.tab.copy()
        self.tab = [None for _ in range(self.w)]
        print("LIMPAR:{}\n\n".format(self.contador_flags))
        self.remapear_valores(tabela_antiga)

    def remapear_valores(self, tabela_antiga):
        for pos, valor in enumerate(tabela_antiga):
            if(valor != None and valor != '#'):      # '#' indica que foi removido um elemento ali
                self.incluir(valor) #Tem que modificar o self.tab ja com a inclusao do valor

    def table_doubling(self):
        tabela_antiga = self.tab.copy()
        m = len(self.tab)
        m *= 2
        self.tab = [None for _ in range(m)]
        self.remapear_valores(tabela_antiga)
        print("DOBRAR TAM:{}\n\n".format(len(self.tab)))

    def halving(self):
        eps=1
        if(len(self.tab)/2>=(eps+1)): #Aqui restringe a tabela a ter pelo menos tamanho 2.
            tabela_antiga = self.tab.copy()
            m = len(self.tab)
            m = int(m/2)
            self.tab = [None for _ in range(m)]
            self.remapear_valores(tabela_antiga)
            print("METADE TAM:{}\n\n".format(len(self.tab)))
            
    def incluir(self, x):
        inserido = False
        h_x = self.funcao_hash(x.chave)
        i = 0
        
        while(not inserido):
            if(i<len(self.tab)):            #pra não percorrer a tabela infinitas vezes.
                pos = (h_x+i)%len(self.tab) #(h[x]+i)mod m
                
                if(self.tab[pos] == None or self.tab[pos] == '#'): 
                    # '#' indica que foi removido um elemento ali
                    if(self.tab[pos] == '#'):
                        self.contador_flags-=1
                    self.tab[pos] = x                    
                    inserido = True
                else:
                    i+=1
                    
            else:
                self.table_doubling() #se a tabela ja estiver cheia, table doubling remapeando valores.
                while(not inserido):
                    if(i<len(self.tab)):            
                        pos = (h_x+i)%len(self.tab) #(h[x]+i)mod m
                        if(self.tab[pos] == None or self.tab[pos] == '#'): 
                            # '#' indica que foi removido um elemento ali
                            if(self.tab[pos] == '#'):
                                self.contador_flags-=1  
                            self.tab[pos] = x
                            inserido = True
                        i+=1

            self.contador_elementos+=1

        
    def buscar(self, x):
        h_x = self.funcao_hash(x)
        pos = (h_x)%len(self.tab)
        i=0
        if(x == None):
            return None
        while(self.tab[pos] != None and self.tab[pos].chave != x and i<len(self.tab)):
            #Pra não percorrer a tabela infinitas vezes.
            pos = (h_x+i)%len(self.tab) #(h[x]+i)mod m
            if(self.tab[pos] != None):
                i += 1
        if(self.tab[pos] != None and self.tab[pos].chave == x):
            return self.tab[pos].valor
        return None
                       

    def remover(self, x): #Implementar o halving (done)

        h_x = self.funcao_hash(x)
        pos = (h_x)%len(self.tab)
        i=0            
        while(self.tab[pos] != None and self.tab[pos].chave != x and i<len(self.tab)):
            pos = (h_x+i)%len(self.tab) #(h[x]+i)mod m
            if(self.tab[pos] != None):
                i += 1
        
        if(self.tab[pos] != None and self.tab[pos].chave == x):
             self.tab[pos] = None #Chave_Valor(float("NaN"),float("NaN")) #
        else:
            return None
        if(self.contador_elementos<=int((1/4)*len(self.tab))):
                self.halving()
        if(self.contador_flags>=(len(self.tab))*(1/4)):
                self.limpar()